In [1]:
!pip install langchain langchain-core langchain-community pydantic

In [2]:
!pip install -U ddgs

In [5]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests


In [ ]:
HUGGINGFACEHUB_API_TOKEN = "hf_xxxxxxxxxxxxx"

In [53]:
@tool
def multiply(a:int, b:int) -> int:
    """given 2 numbers a and b this tool returns their product"""
    return a * b

In [54]:
llm = HuggingFaceEndpoint(
    repo_id = "deepseek-ai/DeepSeek-V3.2",
    huggingfacehub_api_token= HUGGINGFACEHUB_API_TOKEN
)
model = ChatHuggingFace(llm=llm)

In [55]:
llm_with_tools = model.bind_tools([multiply])

In [56]:
llm_with_tools.invoke('Hii how are you?')

AIMessage(content="I'm doing great, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 502, 'total_tokens': 518}, 'model_name': 'deepseek-ai/DeepSeek-V3.2', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c23f5-d1d8-79a2-bd1b-5ae3a09ce4d9-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 502, 'output_tokens': 16, 'total_tokens': 518})

In [57]:
querry = HumanMessage('can you multiply 3 with 1000')

In [58]:
message = [querry]

In [59]:
message

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [60]:
result = llm_with_tools.invoke(message)

In [61]:
message.append(result)

In [62]:
arguments = result.tool_calls[0]

In [63]:
tool_result = multiply.invoke(arguments)

In [64]:
message.append(tool_result)

In [65]:
llm_with_tools.invoke(message).content

"I see you've provided the result of the multiplication: 3000. That's correct! 3 multiplied by 1000 equals 3000.\n\nIs there anything else I can help you with?"